In [1]:
import argparse
import copy
import json
import math
import os
import re
import sys
import numpy as np
from os.path import join
from pathlib import Path
from typing import List, Optional, Union

import requests
import torch
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.nn import Sequential

from tqdm import tqdm
from tqdm.auto import tqdm
from safetensors import safe_open
from safetensors.torch import load_file, save_file

from datasets import load_dataset, Dataset
from itertools import islice
import wandb

from multiprocessing import cpu_count

In [7]:
import transformers
import torch
from transformers import (
    AutoConfig,
    AutoModel,
    AutoModelForCausalLM,
    AutoProcessor,
    AutoTokenizer,
    LlamaTokenizer,
    AutoModelForImageClassification,
    AutoModelForSequenceClassification,
)

In [13]:
from DeepFFNRoBERTa.configuration_roberta import DeepFFNRoBERTaConfig
from DeepFFNRoBERTa.modelling_roberta import (
    DeepFFNRobertaModel,
    DeepFFNRobertaForSequenceClassification,
)

AutoConfig.register("deepffn-roberta", DeepFFNRoBERTaConfig)
AutoModel.register(DeepFFNRoBERTaConfig, DeepFFNRobertaModel)
AutoModelForSequenceClassification.register(DeepFFNRoBERTaConfig, DeepFFNRobertaForSequenceClassification)

In [14]:
config = DeepFFNRoBERTaConfig(
    hidden_size=768,
    intermediate_size=4*768,
    num_hidden_layers=12,
    num_attention_heads=12,
    num_key_value_heads=12,
    num_mlp_layers=1,
)
model = DeepFFNRobertaForSequenceClassification(
    config
)
# model = DeepFFNLlamaForCausalLM(config)
# model = model.to(torch.bfloat16)
# os.makedirs("DeepFFNLLaMA", exist_ok=True)
# model.save_pretrained("DeepFFNLLaMA")
# config.save_pretrained("DeepFFNLLaMA")

In [15]:
model

DeepFFNRobertaForSequenceClassification(
  (roberta): DeepFFNRobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768, padding_idx=1)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): DeepFFNRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x DeepFFNRobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_featur